In [4]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
!pip install -q -U keras-tuner
import keras_tuner as kt

<b> Loading MNIST DATASET </b>

---


<p> assumption - Using MNIST dataset for image classification </p>

In [5]:
(x_train, y_train) , (x_test,y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


<b>Preprocessing</b>
<p> Normalizing all pixel values </p>
<p> converting all values such that they are between 0 and 1</p>

In [6]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

<b> DEFINING MODEL - DENSE NEURAL NETWORK </b>

---

<p>  Building a model for hypertuning, the hyperparameter search space has also been defined here in addition to the model architecture.  </p>

<p> A model builder function has been used to define the image classification model. The model builder function returns a compiled model and uses hyperparameters defined inline to hypertune the model.
 </p>

In [7]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 60-600
  hp_units = hp.Int('units', min_value=60, max_value=600, step=60)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

<b>Instantiate the tuner and perform hypertuning </b>

<p> Instantiate the tuner and perform hypertuning. Hyperband tuner has been used here. </p>

In [9]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3)

<b>RUNNING THE HYPER-PARAMETER SEARCH </b>

In [11]:
tuner.search(x_train, y_train, epochs=50, validation_split=0.2)

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 01m 48s]
val_accuracy: 0.9785833358764648

Best val_accuracy So Far: 0.9803333282470703
Total elapsed time: 00h 31m 30s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 600 and the optimal learning rate for the optimizer
is 0.001.



<b>TRAIN THE MODEL </b>

---
<p> Here the model has been trained on the optimal values of learning rate and number of neurons as found by hyperparameter search by hyperband tuner.</p>

In [12]:
hypermodel = tuner.hypermodel.build(best_hps)
hypermodel.fit(x_train, y_train, epochs=50, validation_split=0.2)

Epoch 1/50
1500/1500 [==============================] - 16s 10ms/step - loss: 0.2160 - accuracy: 0.9362 - val_loss: 0.1128 - val_accuracy: 0.9653
Epoch 2/50
1500/1500 [==============================] - 15s 10ms/step - loss: 0.0859 - accuracy: 0.9729 - val_loss: 0.1025 - val_accuracy: 0.9705
Epoch 3/50
1500/1500 [==============================] - 15s 10ms/step - loss: 0.0550 - accuracy: 0.9827 - val_loss: 0.0842 - val_accuracy: 0.9751
Epoch 4/50
1500/1500 [==============================] - 15s 10ms/step - loss: 0.0373 - accuracy: 0.9882 - val_loss: 0.0776 - val_accuracy: 0.9773
Epoch 5/50
1500/1500 [==============================] - 16s 11ms/step - loss: 0.0277 - accuracy: 0.9906 - val_loss: 0.0902 - val_accuracy: 0.9751
Epoch 6/50
1500/1500 [==============================] - 18s 12ms/step - loss: 0.0230 - accuracy: 0.9927 - val_loss: 0.0835 - val_accuracy: 0.9772
Epoch 7/50
1500/1500 [==============================] - 14s 9ms/step - loss: 0.0153 - accuracy: 0.9950 - val_loss: 0.1015 - 

<b>EVALUATING RESULTS</b>

---
<p> Evaluating the model on the test set. </p>

In [14]:
eval_result = hypermodel.evaluate(x_test, y_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 2s 5ms/step - loss: 0.1913 - accuracy: 0.9788
[test loss, test accuracy]: [0.19131536781787872, 0.9787999987602234]
